In [28]:
import pandas as pd
import os

In [297]:
path = 'lb/'
files_xls = os.listdir(path)
df_list = []
dateparse = lambda x: pd.datetime.strptime(x, '%d/%m/%Y')

for f in files_xls: 
    df_list.append(pd.read_excel("{}{}".format(path,f),converters={'Arrival': str},
                                 parse_dates=False))
final_df = pd.concat(df_list)
final_df = final_df.drop_duplicates()
from datetime import datetime
final_df.loc[final_df['Arrival'].str.len()==19,'Arrival'] = final_df.loc[final_df['Arrival'].str.len()==19,'Arrival'].apply(
    lambda x: datetime.strptime(x, '%Y-%d-%m %H:%M:%S'))
final_df.loc[final_df['Arrival'].str.len()==8,'Arrival'] = final_df.loc[final_df['Arrival'].str.len()==8,'Arrival'].apply(
    lambda x: datetime.strptime(x, '%d/%m/%y'))
final_df['Arrival']= pd.to_datetime(final_df['Arrival'])

final_df = final_df.drop(['B.Code','C.Code','Adults','R.Code','R.Type','Resort',
               'S.Code','Currency','R.Amount','ID','E.Date'],axis=1)
final_df.loc[final_df['Mkt'].isnull(),'Mkt'] = 'BAR'
final_df['Departure'] = final_df['Arrival'] + final_df['Nts'].apply(lambda x: pd.Timedelta(x, unit='d'))

final_df['date_range'] = final_df.apply(lambda row: ','.join([str(row['Arrival']+pd.Timedelta(int(x), unit='d')) for x in range(0,row['Nts'])]),axis =1)
final_df = final_df.reset_index()
final_df.drop('index',inplace=True,axis=1)
final_df = final_df.reset_index()
b = pd.DataFrame(final_df.date_range.str.split(',').tolist(), index=final_df.index).stack()
b = b.reset_index()[['level_0', 0]] # var1 variable is currently labeled 0
b.columns = ['index1', 'checkin'] # renaming var1
b['checkin'] = pd.to_datetime(b['checkin'])
b['checkout'] = b['checkin'] + pd.Timedelta(1, unit='d')
final_df = pd.merge(final_df,b,how='inner',right_on='index1',left_index=True).loc[:,['Book Date','checkin','checkout','Rooms','Mkt','Status']]
final_df.head()

,Book Date,checkin,checkout,Rooms,Mkt,Status
0,2016-01-07 00:00:00,2017-03-09,2017-03-10,1,BAR,CHECKED IN
1,2016-01-07 00:00:00,2017-03-10,2017-03-11,1,BAR,CHECKED IN
2,2016-01-07 00:00:00,2017-03-11,2017-03-12,1,BAR,CHECKED IN
3,2016-01-07 00:00:00,2017-03-12,2017-03-13,1,BAR,CHECKED IN
4,2016-01-07 00:00:00,2017-03-13,2017-03-14,1,BAR,CHECKED IN


In [304]:
segment_mapping = pd.read_excel('lb_segment_mapping.xlsx')
final_df = pd.merge(final_df,segment_mapping,on='Mkt',how='inner')

In [306]:
import numpy as np
final_df.groupby(['name','checkin','Status']).agg({'Rooms':np.sum})

Rooms
name         checkin    Status           
Bar          2017-03-08 CHECKED IN      5
                        DUE OUT         3
             2017-03-09 CHECKED IN     47
                        DUE IN         53
                        DUE OUT         9
             2017-03-10 CHECKED IN     36
                        DUE IN         50
                        DUE OUT         3
                        RESERVED      115
             2017-03-11 CHECKED IN     29
                        DUE IN         34
                        DUE OUT         1
                        RESERVED      166
             2017-03-12 CHECKED IN     18
                        DUE IN         13
                        DUE OUT         1
                        RESERVED      285
             2017-03-13 CHECKED IN      9
                        DUE IN         11
                        RESERVED      373
             2017-03-14 CHECKED IN      4
                        DUE IN          6
                        RESERVED      462
             2017-03-15 CHECKED IN      4
                        DUE IN          5
                        RESERVED      462
             2017-03-16 DUE IN          3
                        RESERVED      478
             2017-03-17 DUE IN          3
                        RESERVED      413
...                                   ...
Travel Agent 2017-12-07 RESERVED        2
             2017-12-08 RESERVED        2
             2017-12-09 RESERVED        2
             2017-12-10 RESERVED        2
             2017-12-22 RESERVED        2
             2017-12-25 RESERVED        4
             2017-12-26 RESERVED        4
             2017-12-27 RESERVED        4
             2017-12-28 RESERVED        2
             2017-12-29 RESERVED        4
             2017-12-30 RESERVED        4
             2017-12-31 RESERVED        2
             2018-01-01 RESERVED        2
             2018-01-02 RESERVED        2
             2018-01-03 RESERVED        4
             2018-01-04 RESERVED        2
             2018-01-05 RESERVED        2
             2018-01-06 RESERVED        1
             2018-01-07 RESERVED        1
             2018-01-08 RESERVED        3
             2018-01-09 RESERVED        2
             2018-01-11 RESERVED        2
             2018-01-12 RESERVED        2
             2018-01-16 RESERVED        1
             2018-01-17 RESERVED        1
             2018-01-18 RESERVED        1
             2018-02-02 RESERVED        2
             2018-02-03 RESERVED        2
             2018-02-04 RESERVED        2
             2018-02-05 RESERVED        2

[1238 rows x 1 columns]

In [309]:
pd.unique(final_df['Book Date'])

array([datetime.datetime(2016, 1, 7, 0, 0),
       datetime.datetime(2016, 2, 8, 0, 0),
       datetime.datetime(2016, 12, 10, 0, 0),
       datetime.datetime(2016, 11, 11, 0, 0),
       datetime.datetime(2016, 11, 22, 0, 0),
       datetime.datetime(2017, 1, 17, 0, 0),
       datetime.datetime(2017, 1, 20, 0, 0),
       datetime.datetime(2017, 1, 24, 0, 0),
       datetime.datetime(2017, 1, 25, 0, 0),
       datetime.datetime(2017, 1, 28, 0, 0),
       datetime.datetime(2017, 3, 2, 0, 0),
       datetime.datetime(2017, 7, 2, 0, 0),
       datetime.datetime(2017, 10, 2, 0, 0),
       datetime.datetime(2017, 11, 2, 0, 0),
       datetime.datetime(2017, 12, 2, 0, 0),
       datetime.datetime(2017, 2, 15, 0, 0),
       datetime.datetime(2017, 2, 17, 0, 0),
       datetime.datetime(2017, 2, 20, 0, 0),
       datetime.datetime(2017, 2, 21, 0, 0),
       datetime.datetime(2017, 2, 22, 0, 0),
       datetime.datetime(2017, 2, 24, 0, 0),
       datetime.datetime(2017, 2, 25, 0, 0),
       date